In [1]:
# # Installation
# !pip install google-vizier[jax]

In [1]:
import numpy as np

: 

In [2]:
from vizier.service import pyvizier as vz
L = 1e-6  # Channel length (m)
VTO = 0.5  # Threshold voltage (Volts)
KP = 50e-6  # Transconductance parameter (A/V^2)
Cox = 2.3e-3  # Oxide capacitance per unit area (F/m^2)
mu_n = KP / Cox  # Mobility (m^2/Vs)

RS = 10e3  # Source resistance (Ohms)
CL = 1e-12  # Load capacitance (F)
gain = 4  # Amplifier gain
f3db = 80e6  # 3dB frequency (Hz)
Omega_3db = 2 * np.pi * f3db  # Angular frequency (rad/s)

Rout = 5e3  # Output resistance (Ohms)
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('Vov', 0.13, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='Id', goal=vz.ObjectiveMetricGoal.MINIMIZE))

def evaluate(Vov: float) -> float:
    # return 1 - 2 * (chocolate - 0.3)**2
    Varx = (1/2) * Omega_3db * gain * CL
    Vary = (2/3) * (L**2) * gain * Omega_3db * RS * (1 / (mu_n  * Rout))
    return (Varx*Vov / (1 - (Vary / Vov)))

In [ ]:
from vizier.service import clients

study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = 'GAUSSIAN_PROCESS_BANDIT'
study_client = clients.Study.from_study_config(study_config, owner='my_name', study_id='ID_obj')

for _ in range(1):
  suggestions = study_client.suggest(count=1)
  for suggestion in suggestions:
    Vov = suggestion.parameters['Vov']
    obj = evaluate(Vov)
    print(f'Iteration {suggestion.id}, suggestion (Vov={Vov:.3f}) led to Id value {obj:.6f}.')
    final_measurement = vz.Measurement({'Id': obj})
    suggestion.complete(final_measurement)

In [ ]:
optimal_trial_client = list(study_client.optimal_trials())[0]
print(optimal_trial_client)
optimal_trial = optimal_trial_client.materialize()
all_trials = [tc.materialize() for tc in study_client.trials()]
#Print optimal Vov and ID from the trials
print(type(optimal_trial.parameters["Vov"]))
# print(f'Optimal Vov: {optimal_trial.parameters["Vov"]:.3f}')
print(dir(optimal_trial.metrics["Id"]))
print(f'Optimal ID: {optimal_trial.metrics["Id"]:.6f}')


In [1]:
# Reset workspace
%reset -f

# From https://github.com/google/vizier/blob/main/README.md
from vizier.service import clients
from vizier.service import pyvizier as vz

# Objective function to maximize.
def evaluate(w: float, x: int, y: float, z: str) -> float:
  return w**2 - y**2 + x * ord(z)

# Algorithm, search space, and metrics.
study_config = vz.StudyConfig(algorithm='DEFAULT')
study_config.search_space.root.add_float_param('w', 0.0, 5.0)
study_config.search_space.root.add_int_param('x', -2, 2)
study_config.search_space.root.add_discrete_param('y', [0.3, 7.2])
study_config.search_space.root.add_categorical_param('z', ['a', 'g', 'k'])
study_config.metric_information.append(vz.MetricInformation('metric_name', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

# Setup client and begin optimization. Vizier Service will be implicitly created.
study = clients.Study.from_study_config(study_config, owner='my_name', study_id='example')
for i in range(10):
  print(f'Iteration {i + 1}')
  suggestions = study.suggest(count=2)
  for suggestion in suggestions:
    params = suggestion.parameters
    objective = evaluate(params['w'], params['x'], params['y'], params['z'])
    suggestion.complete(vz.Measurement({'metric_name': objective}))
    print(f'  Suggestion {suggestion.id}: {params} -> {objective}')

Iteration 1
  Suggestion 1: {'w': 2.5, 'x': 0.0, 'y': 0.3, 'z': 'a'} -> 6.16
  Suggestion 2: {'w': 2.5, 'x': 0.0, 'y': 7.2, 'z': 'g'} -> -45.59
Iteration 2
  Suggestion 3: {'w': 1.3919179414203866, 'x': 1.0, 'y': 0.3, 'z': 'a'} -> 98.84743555564796
  Suggestion 4: {'w': 1.4452676844266819, 'x': -1.0, 'y': 0.3, 'z': 'a'} -> -95.00120132035194
Iteration 3
  Suggestion 5: {'w': 3.7325753623074096, 'x': 1.0, 'y': 7.2, 'z': 'k'} -> 69.09211883530429
  Suggestion 6: {'w': 2.6289150568314303, 'x': 2.0, 'y': 0.3, 'z': 'a'} -> 200.821194376035
Iteration 4
  Suggestion 7: {'w': 4.999999999999999, 'x': 2.0, 'y': 7.2, 'z': 'a'} -> 167.16
  Suggestion 8: {'w': 3.9884730077122423, 'x': 2.0, 'y': 0.3, 'z': 'g'} -> 221.81791693324914
Iteration 5
  Suggestion 9: {'w': 4.999999999999998, 'x': 2.0, 'y': 0.3, 'z': 'k'} -> 238.90999999999997
  Suggestion 10: {'w': 4.999999999999996, 'x': 2.0, 'y': 0.3, 'z': 'a'} -> 218.90999999999997
Iteration 6
  Suggestion 11: {'w': 4.999999999999988, 'x': 2.0, 'y': 0.3,